In [1]:
#import libraries
import numpy as np
from scipy.spatial.transform import Rotation

In [25]:
#Input orbital parameters and constants
a= 66934.2 #km
#h=  19646.8827043885  #km^2/s^2
e=  0.4765126
i=  65.6523  #degrees
Omega=  286.0469  #degrees
omega=  68.9808  #degrees
theta=  0.7 #degrees
#gravitational parameter
mu = 3.986e5 # km^3/s^2

In [26]:
#compute h from a and e
h=np.sqrt(a*mu*(1 - e ** 2))
print(h)

143603.2870383258


In [27]:
#Convert angles from degrees to radians
convFactor=np.pi/180.
i=i*convFactor
Omega=Omega*convFactor
omega=omega*convFactor
theta=theta*convFactor
print('i=',i,'radians')
print('Omega=',Omega)
print('omega=',omega)
print('theta=',theta)

i= 1.1458487965070732 radians
Omega= 4.992460220122967
omega= 1.2039420806597045
theta= 0.012217304763960306


In [28]:
#Define postion and velocity vectors in the perifocal frame from h,e, theta
r_p=h**2/mu/(1+e*np.cos(theta))*np.array((np.cos(theta),np.sin(theta),0))
v_p=mu / h * np.array((-np.sin(theta), e + np.cos(theta),0))
print("r_p=",r_p)
print("norm(r_p)=",np.linalg.norm(r_p))
print("v_p=",v_p)
print("norm(v_p=",np.linalg.norm(v_p))

r_p= [35037.43922929   428.08437241     0.        ]
norm(r_p)= 35040.0542804435
v_p= [-0.03391076  4.09815253  0.        ]
norm(v_p= 4.09829283163296


In [29]:
#Define Rotation matrices
#First one about Z axis an angle Omega
R3W=np.array(((np.cos(Omega),np.sin(Omega),0),(-np.sin(Omega),np.cos(Omega),0),(0,0,1)))
R1i=np.array(((1,0,0),(0,np.cos(i),np.sin(i)),(0,-np.sin(i),-np.cos(i))))
R3w=np.array(((np.cos(omega),np.sin(omega),0),(-np.sin(omega),np.cos(omega),0),(0,0,1)))
print("R3W=",R3W)
print("R1i=",R1i)
print("R3w=",R3w)

R3W= [[ 0.27642411 -0.96103575  0.        ]
 [ 0.96103575  0.27642411  0.        ]
 [ 0.          0.          1.        ]]
R1i= [[ 1.          0.          0.        ]
 [ 0.          0.41227298  0.91106037]
 [ 0.         -0.91106037 -0.41227298]]
R3w= [[ 0.35868078  0.93346028  0.        ]
 [-0.93346028  0.35868078  0.        ]
 [ 0.          0.          1.        ]]


In [30]:
#Multiply matrices to obtain QXx=R1R2R3
QXx=R3w@R1i@R3W
print("QXx=",QXx)

QXx= [[ 0.46899345 -0.23832587  0.85043867]
 [-0.11591835  0.93796475  0.32677984]
 [-0.87556158 -0.25183905 -0.41227298]]


In [31]:
#Compute the QxX transposing QXx
QxX=np.transpose(QXx)
print("QxX=",QxX)

QxX= [[ 0.46899345 -0.11591835 -0.87556158]
 [-0.23832587  0.93796475 -0.25183905]
 [ 0.85043867  0.32677984 -0.41227298]]


In [32]:
#To follow the algorithm we defined in class,
#we need to use the column vector, this is the main difference
#in the matrix multiplication

#reshape, allow to convert the column vector we defined for the 
#perifocal frame into a column. The (-1,1) do the conversion from 
#row to column; if you need from column to row, use reshape with (1,-1)
r_pc=r_p.reshape(-1,1)
#We perform the matrix computation as always
r=QxX@r_pc
print("r=",r," km")
print("r=",r.reshape(1,-1), " km")
#convert the perifocal velocity from row to column vector 
v_pc=v_p.reshape(-1,1)
#perform the multiplication as always
v=QxX@v_pc
print("v=",v, " km/s")
print("v=",v.reshape(1,-1), " km/s")

r= [[16382.70656947]
 [-7948.80001338]
 [29937.08248615]]  km
r= [[16382.70656947 -7948.80001338 29937.08248615]]  km
v= [[-0.49095502]
 [ 3.85200443]
 [ 1.3103546 ]]  km/s
v= [[-0.49095502  3.85200443  1.3103546 ]]  km/s


In [33]:
R=Rotation.from_euler("ZXZ",[-omega,-i,-Omega])
print("R=",R.as_matrix())
print("QXx=",QXx)

R= [[ 0.46899345 -0.23832587  0.85043867]
 [-0.11591835  0.93796475  0.32677984]
 [-0.87556158 -0.25183905  0.41227298]]
QXx= [[ 0.46899345 -0.23832587  0.85043867]
 [-0.11591835  0.93796475  0.32677984]
 [-0.87556158 -0.25183905 -0.41227298]]


In [34]:
r=r_p@R.as_matrix()
print(r," km")

[16382.70656947 -7948.80001338 29937.08248615]  km


In [35]:
v=v_p@R.as_matrix()
print(v, " km/s")

[-0.49095502  3.85200443  1.3103546 ]  km/s


In [14]:
r=r_p@QXx
print(r," km")
v=v_p@QXx
print(v," km/s")

[5776.4114103  2358.21008327 2358.21008327]  km
[-3.90249889  4.87223198  4.87223198]  km/s
